In [ ]:
import torch
from monai.transforms import Resize
from models import InversionTimeLSTM, FeatureExtractor
from utils import load_parameters
import json

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

In [ ]:
def load_model(p):
    with open(f'{p}/config.json', 'r') as f:
        config = json.load(f)
    
    feature_extractor = FeatureExtractor(
        config['name'], dropout=config['dropout'][0], dim_spatial=config['dim_spatial'], pooling=config['pooling'], device=device
    )
    model = InversionTimeLSTM(
        feature_extractor, num_layers=config['num_layers'], hidden_size=config['hidden_size'], dropout=config['dropout'][1],
        input_as_window=False
    ).to(device)
    
    load_parameters(model, optimiser=None, directory=p, name=None, feature_extractor_only=False, load_index=-1)
    
    return model, config

In [ ]:
model, config = load_model('./model')

In [ ]:
# Retrieve image array x, then resize to 256 by 256
x = ... # (frame, M, N)
x = Resize((256, 256))(x)

In [ ]:
# Infer 
model.infer(x, standardise=True)